Comparing_Similarity.ipynb program is for applying the Similarity_btw_emails program for recived email with all the other email 

In [1]:
import nltk
import os
from email.parser import Parser
import math
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import gensim
from gensim.models.doc2vec import TaggedDocument

In [2]:
rootdir1 = r"enron_mail_20150507.tar\enron_mail_20150507\maildir\lay-k\all_documents\33_"
with open(rootdir1,"r") as f:
    data1 = f.read()
email1 = Parser().parsestr(data1)

FileNotFoundError: [Errno 2] No such file or directory: 'enron_mail_20150507.tar\\enron_mail_20150507\\maildir\\lay-k\\all_documents\\33_'

In [3]:
rootdir = "enron_mail_20150507.tar\\enron_mail_20150507\\maildir\\lay-k\\elizabeth\\"

creating class for the function the checking similarity btw documents

In [4]:
class Similarity:
    stopwords_en = stopwords.words("english")
    def __init__(self,rec_mail,email_dir):        
        self.rec_mail = rec_mail
        self.similirity_list = list()
        self.rootdir = email_dir
        self.top5 = list()
        self.email_body = ""
    
    def email_body_single(self,inputfile):
        with open(inputfile,"r") as f:
            data=f.read()

        email = Parser().parsestr(data)

        self.email_body= self.email_body + (email.get_payload().replace("\n",""))

    def email_body_sum(self):
        for directory, subdirectory,filenames in os.walk(rootdir):
            for filename in  filenames:
                try:
                    self.email_body_single(os.path.join(directory,filename))
                except:
                    print(os.path.join(directory,filename))

    def preprocessing(self,raw):
        wordlist = nltk.word_tokenize(raw)
        text = [w.lower() for w in wordlist if w not in self.stopwords_en]
        return text

    def email_analyse(self,inputfile):

        with open(inputfile,"r") as f:
            data2=f.read()
    
        email2 = Parser().parsestr(data2)


        text1 = self.preprocessing(self.rec_mail.replace("\n",""))
        text2 = self.preprocessing(email2.get_payload().replace("\n",""))

        word_set = set(text1).union(set(text2))
        word_set_idf = set(text1).union(set(text2))#set(self.email_body)

        freqd_text1 = FreqDist(text1)
        text1_count_dict = dict.fromkeys(word_set,0)
        for word in text1:
            text1_count_dict[word] = freqd_text1[word]

        freqd_text2 = FreqDist(text2)
        text2_count_dict = dict.fromkeys(word_set,0)
        for word in text2:
            text2_count_dict[word] = freqd_text2[word]
        
        freqd_text1 = FreqDist(text1)
        text1_length = len(text1)
        text1_tf_dict = dict.fromkeys(word_set,0)
        for word in text1:
            text1_tf_dict[word] = freqd_text1[word]/text1_length

        freqd_text2 = FreqDist(text2)
        text2_length = len(text2)
        text2_tf_dict = dict.fromkeys(word_set,0)
        for word in text2:
            text2_tf_dict[word] = freqd_text2[word]/text2_length
        
        text12_idf_dict = dict.fromkeys(word_set_idf,0)
        text12_length = 2

        for word in text12_idf_dict.keys():
            if word in text1:
                text12_idf_dict[word] += 1
            if word in text2:
                text12_idf_dict[word] += 1
        
        for word, val in text12_idf_dict.items():
            if val!=0:
                text12_idf_dict[word] = 1 + math.log(text12_length/(float(val)))
        
        text1_tfidf_dict = dict.fromkeys(word_set,0)
        for word in text1:
            text1_tfidf_dict[word] = (text1_tf_dict[word]) * (text12_idf_dict[word])

        text2_tfidf_dict = dict.fromkeys(word_set,0)
        for word in text2:
            text2_tfidf_dict[word] = (text2_tf_dict[word]) * (text12_idf_dict[word])
        
        taggeddocs = list()
        doc1 = TaggedDocument(words=text1,tags= ["Mail_1"])
        taggeddocs.append(doc1)
        doc2 = TaggedDocument(words=text2,tags= ["Mail_2"])
        taggeddocs.append(doc2)

        model = gensim.models.Doc2Vec(taggeddocs,dm=0,alpha=0.025,vector_size=20,min_alpha=0.025,min_count=0)
        token_count = sum([len(sentence) for sentence in taggeddocs])
        for epoch in range(80):
            if epoch % 20 ==0:
                print("Now training epoch %s" %epoch)
            model.train(taggeddocs,total_examples = token_count, epochs = 80)
            model.alpha -= 0.002
            model.min_alpha = model.alpha
        
        v1 = list(text1_tfidf_dict.values())
        v2 = list(text2_tfidf_dict.values())

        similarity = 1 - nltk.cluster.cosine_distance(v1,v2)
        self.similirity_list.append(similarity)
        print("Similarity Index: {:4.2f} %".format(similarity*100))



    def check_similarity(self):
        for directory, subdirectory,filenames in os.walk(self.rootdir):
            for filename in  filenames:
                # try:
                self.email_analyse(os.path.join(directory,filename))
                # except:
                #     print("Error: ",os.path.join(directory,filename))
        
    def display_similarities(self):
        for i in range (5):
            self.top5.append(max(self.similirity_list))
            self.similirity_list.remove(max(self.similirity_list))
        print(self.similirity_list)

In [5]:
Obj = Similarity(email1.get_payload(),rootdir)

In [6]:
Obj.email_body_sum()

In [7]:
Obj.check_similarity()

Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 17.26 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 13.69 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 16.33 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 16.98 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 15.94 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 5.78 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 23.85 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity Index: 9.13 %
Now training epoch 0
Now training epoch 20
Now training epoch 40
Now training epoch 60
Similarity 

In [8]:
Obj.display_similarities()

[0.17255341069937424, 0.1368944884372253, 0.1632895635946925, 0.16983252667167914, 0.15938086247429695, 0.05776345370057989, 0.23849064124259867, 0.09131335472552704, 0.06314764008507079, 0.21869178760314223, 0.16298013416373291, 0.18474541061907335, 0.00714036020633646, 0.1136480350397262, 0.14540446962568, 0.14768816757350112, 0.035521337988735024, 0.076408851366907, 0.034095580742622644, 0.11094838890413594, 0.21005735033097683, 0.06822070022727211, 0.12569659144486045, 0.1392510950843624, 0.15415709172554248, 0.1662684963180796, 0.14540057439964882, 0.0, 0.021858557192827255, 0.1438185621390231]
